### EDA Lecture - Finals Assignment

#### Temperature Sensor Data Outlier Detection

##### Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

##### Load & prepare the data

In [ ]:
df_set_temp = pd.read_csv(filepath_or_buffer='./Data/Setpoint_LazienkaGorna.csv')
df_measured_temp = pd.read_csv(filepath_or_buffer='./Data/Temperatura_LazienkaGorna.csv')
df_outside_temp = pd.read_csv(filepath_or_buffer='./Data/TemperaturaZewnetrzna.csv')

In [ ]:
print(df_set_temp.head())
print(df_measured_temp.head())
print(df_outside_temp.head())

In [ ]:
# Convert time to human-readable datetime

df_set_temp['time'] = pd.to_datetime(df_set_temp['time'])
df_set_temp.head()

In [ ]:
df_measured_temp['time'] = pd.to_datetime(df_set_temp['time'])
df_outside_temp['time'] = pd.to_datetime(df_set_temp['time'])

In [ ]:
# Drop any rows with NaN values

df_measured_temp = df_measured_temp.dropna(subset=['value', 'time'])
df_set_temp = df_set_temp.dropna(subset=['value', 'time'])
df_outside_temp = df_outside_temp.dropna(subset=['value', 'time'])

##### Visualize the data

In [ ]:
fix, axes = plt.subplots(nrows=3, ncols=1, figsize=(15, 5))

axes[0].plot(df_set_temp['time'], df_set_temp['value'], label='Set Temp')
axes[0].set_title('Set Temp')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()

axes[1].plot(df_measured_temp['time'], df_measured_temp['value'], label='Measured Temp')
axes[1].set_title('Measured Temp')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].legend()

axes[2].plot(df_outside_temp['time'], df_outside_temp['value'], label='Outside Temp')
axes[2].set_title('Outside Temp')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Value')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Redraw the plots with a smaller timeframe

fix, axes = plt.subplots(nrows=3, ncols=1, figsize=(15, 5))

axes[0].plot(df_set_temp['time'][:8000], df_set_temp['value'][:8000], label='Set Temp')
axes[0].set_title('Set Temp')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()

axes[1].plot(df_measured_temp['time'][:8000], df_measured_temp['value'][:8000], label='Measured Temp')
axes[1].set_title('Measured Temp')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].legend()

axes[2].plot(df_outside_temp['time'][:8000], df_outside_temp['value'][:8000], label='Outside Temp')
axes[2].set_title('Outside Temp')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Value')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Add lines to help spot trend patterns in the data points

fix, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 10))

for year in range(2020, 2024):
    # Line plots for year 2020
    if year == 2020:
        for month in range(9, 13):
            for ax in axes:
                ax.axvline(datetime(year, month, 1), color='k', linestyle='--', alpha=0.5)
    # Line plots for year 2023
    elif year == 2023:
        for month in range(1, 4):
            for ax in axes:
                ax.axvline(datetime(year, month, 1), color='k', linestyle='--', alpha=0.5)
    # Line plots for other years
    else:
        for month in range(1, 13):
            for ax in axes:
                ax.axvline(datetime(year, month, 1), color='k', linestyle='--', alpha=0.5)

axes[0].plot(df_set_temp['time'], df_set_temp['value'], label='Set Temp')
axes[0].set_title('Set Temp')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()

axes[1].plot(df_measured_temp['time'], df_measured_temp['value'], label='Measured Temp')
axes[1].set_title('Measured Temp')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].legend()

axes[2].plot(df_outside_temp['time'], df_outside_temp['value'], label='Outside Temp')
axes[2].set_title('Outside Temp')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Value')
axes[2].legend()

plt.tight_layout()
plt.show()

##### Find min / max values from the available data

In [ ]:
df_sets = [df_set_temp, df_measured_temp, df_outside_temp]

for dataset in df_sets:
    print(f'Data: {dataset["name"][0]}')
    print(f'MIN: {dataset["value"].min()}')
    print(f'MAX: {dataset["value"].max()}')
    print(f'Median: {dataset["value"].median()}')
    print(f'Mean: {dataset["value"].mean()}')
    print()

In [ ]:
# Find the standard deviation value for the dataset values

set_temp_deviation = df_set_temp['value'].std()
print(f'Set temp deviation: {set_temp_deviation}')

measured_temp_deviation = df_measured_temp['value'].std()
print(f'Measured temp deviation: {measured_temp_deviation}')

outside_temp_deviation = df_outside_temp['value'].std()
print(f'Outside temp deviation: {outside_temp_deviation}')

In [ ]:
# Find the bounds for the outliers in data via IQR

def calculate_outlier_threshold(data):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 4 * iqr
    upper_bound = q3 + 4 * iqr
    return lower_bound, upper_bound

In [ ]:
lower_bound, upper_bound = calculate_outlier_threshold(df_set_temp['value'])
print(f'Outlier Threshold for the set temp: below {lower_bound} or above {upper_bound}')

lower_bound, upper_bound = calculate_outlier_threshold(df_measured_temp['value'])
print(f'Outlier Threshold for the measured temp: below {lower_bound} or above {upper_bound}')

lower_bound, upper_bound = calculate_outlier_threshold(df_outside_temp['value'])
print(f'Outlier Threshold for the outside temp: below {lower_bound} or above {upper_bound}')

##### Look for outliers in data

In [ ]:
from scipy.stats import zscore

def run_zscore(dataset):
    dataset['z_score'] = zscore(dataset['value'])
    anomalies = dataset[abs(dataset['z_score']) > 4]

    plt.figure(figsize=(20, 8))
    plt.plot(dataset['time'], dataset['value'])
    plt.scatter(anomalies['time'], anomalies['value'], color='red', label='Anomalies')
    plt.legend()
    plt.show()

In [ ]:
run_zscore(df_measured_temp)

In [ ]:
run_zscore(df_set_temp)

In [ ]:
run_zscore(df_outside_temp)

In [ ]:
def run_iqr(dataset):
    q1 = dataset['value'].quantile(0.25)
    q3 = dataset['value'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 5 * iqr
    upper_bound = q3 + 5 * iqr

    anomalies = dataset[(dataset['value'] < lower_bound) | (dataset['value'] > upper_bound)]

    plt.figure(figsize=(20, 8))
    plt.plot(df_measured_temp['time'], dataset['value'])
    plt.scatter(anomalies['time'], anomalies['value'], color='red', label='Anomalies')
    plt.legend()
    plt.show()

In [ ]:
run_iqr(df_measured_temp)

In [ ]:
run_iqr(df_set_temp)

In [ ]:
run_iqr(df_outside_temp)

In [ ]:
from sklearn.ensemble import IsolationForest

def run_isolation_forest(dataset):
    model = IsolationForest(contamination=0.0025)
    dataset['anomaly'] = model.fit_predict(dataset[['value']])
    anomalies = dataset[dataset['anomaly'] == -1]

    plt.figure(figsize=(20, 8))
    plt.plot(dataset['time'], dataset['value'])
    plt.scatter(anomalies['time'], anomalies['value'], color='red', label='Anomalies')
    plt.legend()
    plt.show()

In [ ]:
run_isolation_forest(df_measured_temp)

In [ ]:
run_isolation_forest(df_set_temp)

In [ ]:
run_isolation_forest(df_outside_temp)

In [ ]:
from statsmodels.tsa.seasonal import STL

# Set sampling period: 1 reading every 5 minutes - 12 samples per hour - 288 samples per day

period = 288

# Dictionary to store outlier detection results for each dataset

results_dict = {}

# Dictionary to store anomalies found for each dataset

anomaly_dict = {}

datasets = {
    'Measured temp': df_measured_temp,
    'Set temp': df_set_temp,
    'Outside temp': df_outside_temp
}

In [ ]:
def run_STL_model(dataset_name, period):
    dataset = datasets[dataset_name]
    data = dataset['value']

    stl = STL(data, period=period)
    result = stl.fit()

    results_dict[dataset_name] = result

    seasonal, trend, residual = result.seasonal, result.trend, result.resid

    # Plot the result charts

    plt.figure(figsize=(16, 8))

    plt.subplot(4,1,1)
    plt.plot(data)
    plt.title(f'Original {dataset_name} Series', fontsize=16)

    plt.subplot(4,1,2)
    plt.plot(trend)
    plt.title(f'{dataset_name} Trend', fontsize=16)

    plt.subplot(4,1,3)
    plt.plot(seasonal)
    plt.title(f'{dataset_name} Seasonal', fontsize=16)

    plt.subplot(4,1,4)
    plt.plot(residual)
    plt.title(f'{dataset_name} Residual', fontsize=16)

    plt.tight_layout()
    plt.show()

In [ ]:
for dataset_name in datasets:
    run_STL_model(dataset_name, period)

In [ ]:
def find_STL_outliers(dataset_name, results_dict):
    dataset = datasets[dataset_name]
    data = dataset['value']
    results = results_dict[dataset_name]
    residual = results.resid

    estimated = results.trend + results.seasonal

    plt.figure(figsize=(12, 4))
    plt.plot(data)
    plt.plot(estimated)
    plt.title(f'{dataset_name} Estimate vs Original difference')
    plt.show()

    resid_mu = residual.mean()
    resid_dev = residual.std()

    lower = resid_mu - (6.6 * resid_dev)
    upper = resid_mu + (6.6 * resid_dev)

    anomalies = dataset[(residual < lower) | (residual > upper)]
    anomaly_dict[dataset_name] = anomalies

    plt.figure(figsize=(12, 4))
    plt.plot(results.resid)
    plt.fill_between([datetime(2020, 9, 1), datetime(2023, 4, 1)], lower, upper, color='g', alpha=0.25, linestyle='--', linewidth=2)
    plt.xlim(datetime(2020, 9, 1), datetime(2023, 4, 1))
    plt.show()

    plt.figure(figsize=(12, 4))
    plt.plot(data)

    for year in range (2020, 2023):
        plt.axvline(datetime(year, 1, 1), color='k', linestyle='--', alpha=0.25)
    
    plt.scatter(anomalies.index, anomalies['value'], color='r', marker='D')
    plt.show()

In [ ]:
for dataset_name in datasets:
    find_STL_outliers(dataset_name, results_dict)

In [ ]:
anomaly_dict['Measured temp']

In [ ]:
anomaly_dict['Set temp']

In [ ]:
anomaly_dict['Outside temp']